**21/11/2021**

* Angel Luna                        A01177358
* Jesús David Guajardo Ovalle       A01283614
* Sebastián Fernández del Valle     A01720716 
* Luis Carlos Larios Cota           A00826904

**INTRUCCIONES**

Realiza la siguiente simulación:
* Inicializa las posiciones iniciales de las K cajas. Todas las cajas están a nivel de piso, es decir, no hay pilas de cajas.
* Todos los agentes empiezan en posición aleatorias vacías.
* Se ejecuta el tiempo máximo establecido.

Deberás recopilar la siguiente información durante la ejecución:
* Tiempo necesario hasta que todas las cajas están en pilas de máximo 5 cajas.
* Número de movimientos realizados por todos los robots.


In [ ]:
!pip install mesa
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

# mathplotlib lo usamos para graficar/visualizar como evoluciona el autómata celular.
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime
import random

In [ ]:
def get_grid(model):
  '''
  Esta es una función auxiliar que nos permite guardar el grid para cada uno de los agentes.
  '''
  grid = np.zeros((model.grid.width, model.grid.height))
  for cell in model.grid.coord_iter():
    cell_content, x, y = cell
    for obj in cell_content:
      if isinstance(obj, RobotCajas):
        grid[x][y] = 3
      elif isinstance(obj, Celda):
        grid[x][y] = obj.estado
  return grid

class RobotCajas(Agent):
  '''
  Representa a un robot de cajas que puede llegar a tenr hasta cinco cajas.
  '''
  def __init__(self, unique_id, model):
    super().__init__(unique_id, model)
    self.cajas = 0
    self.sig_pos = None
  
  def step(self):
    vecinos = self.model.grid.get_neighbors(
      self.pos,
      moore=True,
      include_center=True)
    
    # maquina de estados
    for vecino in vecinos:
      if isinstance(vecino, Celda) and self.pos == vecino.pos:
        if vecino.estado == 1:
          vecino.sig_estado = 0
          self.sig_pos = self.pos
          self.cajas = self.cajas + 1
          if self.cajas == 5:
            print('PILA COMPLETADA') # SI SE COMPLETAN LAS 5 CAJAS EMPIEZA DESDE CERO.
            vecino.sig_estado = 2
            self.cajas = 0
        else:
          vecinos_2 = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
          if vecino.sig_estado == 2: # CELDAS QUE SON PILAS DE 5 SE MANTIENEN IGUAL.
            vecino.sig_estado = 2
          else:
            vecino.sig_estado = 0
          self.sig_pos = self.random.choice(vecinos_2)
        break
    # termina maquina de estados

  def advance(self):
    '''
    Define el nuevo estado calculado del método step.
    '''
    vecinos = self.model.grid.get_neighbors(
      self.pos,
      moore=True,
      include_center=True)
    for vecino in vecinos:
      if isinstance(vecino, Celda) and self.pos == vecino.pos:
        vecino.estado = vecino.sig_estado
        break
    self.model.grid.move_agent(self, self.sig_pos)

class Celda(Agent):
  # 1 = Caja
  # 0 = No Caja
  def __init__(self, unique_id, model, estado):
    super().__init__(unique_id, model)
    self.pos = unique_id
    self.estado = estado
    self.sig_estado = None

class Habitacion(Model):
  '''
    Define el modelo del programa.
  '''
  def __init__(self, M, N, num_agentes, num_cajas):
    self.num_agentes = num_agentes
    self.num_cajas = num_cajas
    self.grid = MultiGrid(M, N, False)
    self.schedule = SimultaneousActivation(self)

    # Celdas
    num_celdas_con_caja = num_cajas
    for (content, x, y) in self.grid.coord_iter():
      num = random.randint(0,1)
      if num == 1 and num_celdas_con_caja > 0:
        a = Celda((x,y), self, 1)
        num_celdas_con_caja -= 1
      else:
        a = Celda((x,y), self, 0)
      self.grid.place_agent(a, (x, y))
      self.schedule.add(a)

    # Agentes Robot
    for id in range(num_agentes):
      r = RobotCajas(id, self)
      self.grid.place_agent(r, (M-2, N-2))
      self.schedule.add(r)

    self.datacollector = DataCollector(
      model_reporters={"Grid": get_grid})
    
  def step(self):
    '''
        En cada paso el colector tomará la información que se definió y almacenará el grid para luego graficarlo.
    '''
    self.datacollector.collect(self)
    self.schedule.step()

  def todoApilado(self):
    for (content, x, y) in self.grid.coord_iter():
      for obj in content:
        if isinstance(obj, Celda) and obj.estado == 1:
          return False
    print()
    for (content, x, y) in self.grid.coord_iter():
      for obj in content:
        if isinstance(obj, RobotCajas):
          print('Robot: ', obj.unique_id, ' Cajas: ', obj.cajas)
    return True


In [ ]:
M = 10
N = 10
num_agentes = 5
num_cajas = 30
tiempo_maximo = 0.5

model = Habitacion(M, N, num_agentes, num_cajas)
start_time = time.time()
while((time.time() - start_time) < tiempo_maximo and not model.todoApilado()):
  model.step()
print('La informacion superior son las cajas pendientes sobre los robots.')
print()
print('Tiempo de ejecución: ', str(datetime.timedelta(seconds=(time.time() - start_time))))


PILA COMPLETADA
PILA COMPLETADA
PILA COMPLETADA
PILA COMPLETADA

Robot:  2  Cajas:  4
Robot:  1  Cajas:  0
Robot:  4  Cajas:  3
Robot:  3  Cajas:  0
Robot:  0  Cajas:  3
La informacion superior son las cajas pendientes sobre los robots.

Tiempo de ejecución:  0:00:00.086129


In [ ]:
all_grid = model.datacollector.get_model_vars_dataframe()
# print(all_grid.to_string())

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(all_grid.iloc[0][0], cmap=plt.cm.binary)

def animate(i):
  patch.set_data(all_grid.iloc[i][0])

anim = animation.FuncAnimation(fig, animate, frames=len(all_grid))


In [ ]:
anim